In [12]:
# Autor: Dawid Żamojda
# Program powstał na Wydziale Informatyki Politechniki Białostockiej
from csv import DictReader
import osmnx as ox
import networkx as nx
import numpy as np
from datetime import datetime
# W celu poprawnego działania programu należy zainstalować odpowiednie biblioteki
# Folder z programem zawiera plik requirements.txt z zapisnymi bibliotekami
# W celu zainstalowania potrzebnych bibliotek należy użyc komendy 'pip install -r requirements.txt' 

In [13]:
file = open("punkty10.txt", 'r')
reader = DictReader(file)
points = list()

for dictionary in reader:
    points.append(dictionary)
print(points)

[{'adres': 'adres1', 'y': '23.141401', 'x': '53.142928'}, {'adres': 'adres2', 'y': '23.150168', 'x': '53.125021'}, {'adres': 'adres3', 'y': '23.094750', 'x': '53.157217'}, {'adres': 'adres4', 'y': '23.095895', 'x': '53.149964'}, {'adres': 'adres5', 'y': '23.198124', 'x': '53.106625'}, {'adres': 'adres6', 'y': '23.175494', 'x': '53.125748'}, {'adres': 'adres7', 'y': '23.147174', 'x': '53.117526'}, {'adres': 'adres8', 'y': '23.135313', 'x': '53.13227'}, {'adres': 'adres9', 'y': '23.165609', 'x': '53.131947'}, {'adres': 'adres10', 'y': '23.119271', 'x': '53.135303'}]


In [14]:
ox.settings.log_console = True
ox.settings.use_cache = True

In [15]:
miejsce = "Białystok, Polska"
tryb = "drive"
kryterium = "length"

In [16]:
graph = ox.graph_from_place(miejsce, network_type=tryb )

In [17]:
distance_matrix = np.empty((len(points),len(points)))

In [18]:
for i in range(len(points)):
    origin_node = ox.nearest_nodes(graph, float(points[i]['y']), float(points[i]['x']))
    for j in range(len(points)):
        destination_node = ox.nearest_nodes(graph, float(points[j]['y']), float(points[j]['x']))
        distance_matrix[i][j] = nx.shortest_path_length(graph, origin_node, destination_node, weight = kryterium)
#distance_matrix

In [19]:
import gen_algorithms as ga

In [20]:
#Hyperparameters:
number_of_generations = 10
population_size = 100
number_of_parents = 60
length = len(points)
beginning = 1
number_of_offspring = population_size - number_of_parents
mutation_rate = 10 #%

In [21]:
#Populacja początkowa:
start_time = datetime.now()
population = []
i=0
while i < population_size:
    ind = ga.create_individual(beginning, length)
    if ind not in population:
        population.append(ind)
        i+=1
best_individuals=[]

#Kolejne iteracje algorytmu genetycznego
for generation in range(number_of_generations):

    fitness_list=[]
    for indiv in population:
        fitness_list.append(ga.calculate_fitness(indiv, distance_matrix, length))
    best_individual = min(fitness_list)
    best_individuals.append(best_individual)

    parents = ga.select_parents(population, fitness_list, number_of_parents)

    children = ga.crossover(parents, number_of_offspring, length)

    for child in children:
        ga.mutation(child, mutation_rate)
    
    population = parents + children
end_time = datetime.now()

In [22]:
# Wypisywanie wyników do pliku wyjściowego
duration = end_time - start_time
print(f'Duration: {duration} \n\n')
difference = best_individuals[0] - best_individuals[-1]
improvement = (difference/best_individuals[0])*100
print(f'first generation: {best_individuals[0]}, last_generation: {best_individuals[-1]}')
print(f'difference: {difference}, improvement: {improvement} %')

best = min(fitness_list)
best_route_index = fitness_list.index(best)
best_route = population[best_route_index]
f=open("Solution.txt", 'w')
f.write(f'Duration: {duration}\n')
f.write(f'first generation: {best_individuals[0]}, \nlast_generation: {best_individuals[-1]}\n')
f.write(f'difference: {difference}, \nimprovement: {improvement} %\n')
f.write(f'best route: {best_route}\n\n')

Duration: 0:00:00.005999 


first generation: 30432.968999999997, last_generation: 28136.593999999997
difference: 2296.375, improvement: 7.545681790035013 %
